In [2]:
import pandas as pd
from ast import literal_eval
import nltk
import wordcloud
import inflect

#读取信息

In [3]:
df = pd.read_csv(r'F:\work\Image_emotion_analysis\artemis-master\RAIVS_out\step1\artemis_preprocessed.csv')
text_list = df['utterance'].tolist()
text = text_list[0]
text

'She seems very happy in the picture, and you want to know what what is behind the smile.'

#词性标注，获取名词

In [4]:
def get_NNwords(text):
    #获取名词
    # 分割句子
    words = nltk.word_tokenize(text)
    print(words)
    # 词性标注
    taged_sent = nltk.pos_tag(words)
    print(taged_sent)

    #输出名词https://blog.csdn.net/qq_36652619/article/details/77252497
    NN = ['NN','NNS','NNP','NNPS']
    NN_words = []
    for w,l in taged_sent:
        if l in NN:
            print(w)
            NN_words.append(w)
    return NN_words 
NN_words=get_NNwords('he doesn\'t say, I know, I am a sick cat!')   

['he', 'does', "n't", 'say', ',', 'I', 'know', ',', 'I', 'am', 'a', 'sick', 'cat', '!']
[('he', 'PRP'), ('does', 'VBZ'), ("n't", 'RB'), ('say', 'VB'), (',', ','), ('I', 'PRP'), ('know', 'VBP'), (',', ','), ('I', 'PRP'), ('am', 'VBP'), ('a', 'DT'), ('sick', 'JJ'), ('cat', 'NN'), ('!', '.')]
cat


In [4]:

'''
NN_words = []
for text in text_list:
    NN_words=NN_words+get_NNwords(text) 
'''
f = open('NN_words.txt','r')
content=f.read()
f.close()
NN_words = literal_eval(content)
    


In [5]:
#抽象字典准备：
brm_file = r'F:\work\Image_emotion_analysis\artemis-master\RAIVS\scripts\test\Concreteness_ratings_Brysbaert_et_al_BRM.xlsx'
brm_data = pd.read_excel(brm_file,sheet_name=0)
brm_data.Word = brm_data.Word.apply(lambda x: str(x).lower()) 
brm_data = brm_data[brm_data.Dom_Pos != 'Article']
brm_data = brm_data[['Word', 'Conc.M']]
brm_data.columns = ['word', 'concreteness']
word_to_concreteness = dict(zip(brm_data.word, brm_data.concreteness))
word_to_concreteness

{'roadsweeper': 4.85,
 'traindriver': 4.54,
 'tush': 4.45,
 'hairdress': 3.93,
 'pharmaceutics': 3.77,
 'hoover': 3.76,
 'shopkeeping': 3.18,
 'pushiness': 2.48,
 'underdevelop': 2.37,
 'tirelessness': 2.28,
 'oldfashioned': 2.26,
 'wellmannered': 2.25,
 'dismissiveness': 1.83,
 'spitefulness': 1.8,
 'untruthfulness': 1.73,
 'dispiritedness': 1.56,
 'sled': 5.0,
 'plunger': 4.96,
 'human': 4.93,
 'waterbed': 4.93,
 'cymbal': 4.92,
 'ginger': 4.92,
 'bobsled': 4.9,
 'cardboard': 4.9,
 'olive': 4.9,
 'dogsled': 4.89,
 'rubber': 4.86,
 'soybean': 4.82,
 'tangerine': 4.81,
 'headrest': 4.8,
 'eucalyptus': 4.77,
 'saltwater': 4.77,
 'armrest': 4.76,
 'paramedic': 4.74,
 'liquid': 4.72,
 'billfold': 4.71,
 'canine': 4.71,
 'flowerbed': 4.71,
 'soy': 4.7,
 'bald': 4.69,
 'lilac': 4.69,
 'hemorrhoid': 4.68,
 'orange': 4.66,
 'arachnid': 4.65,
 'underarm': 4.63,
 'barefoot': 4.62,
 'bearded': 4.62,
 'thyroid': 4.61,
 'wooden': 4.61,
 'sleeveless': 4.6,
 'concrete': 4.59,
 'panty': 4.59,
 'pregn

#获得抽象分数字典

In [6]:

#将名词复数变成名词单数
#p.singular_noun(word)返回名词单数/如果本身是名词单数，则返回false
'''
复数处理之前：
missed_words: 312868 28191
abstract_words: 1258992 12559
复数处理之后：
missed_words: 134645 22932
abstract_words: 1437215 12182
'''
import nltk
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))

NN_words_new = []
p = inflect.engine()
for word in NN_words:
    result = p.singular_noun(word)
    if result not in stopset:
        if result==False:
            NN_words_new.append(word)
        else:
            NN_words_new.append(result)

In [7]:
len(NN_words_new)==len(NN_words)

False

#有多少词有抽象分数？有多少没有抽象分数？

In [8]:
missed_words=[]
abstract_words=[]
for w in NN_words_new:
    if w in word_to_concreteness:
        abstract_words.append(w)
    else:
        missed_words.append(w)
print('missed_words:',len(missed_words),len(set(missed_words)))
print('abstract_words:',len(abstract_words),len(set(abstract_words)))

data = missed_words
f = open('missed_words.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

data = abstract_words
f = open('abstract_words.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

missed_words: 134611 22928
abstract_words: 1435317 12156


#获取1000个最抽象的词汇vs 1000个最具体的词汇

In [9]:
#abstract_words与抽象评分
abstract_abs_dict={}
for word in set(abstract_words):
    abstract_abs_dict[word]=word_to_concreteness[word]

#根据value从小到大排序
abstract_abs_dict = sorted(abstract_abs_dict.items(), key=lambda x: x[1])

#抽象评分最小的1000个词
abstract_1000 = []
specific_1000=[]
for word,score in abstract_abs_dict[:1000]:
    abstract_1000.append(word)
for word,score in abstract_abs_dict[-1000:]:
    specific_1000.append(word)

In [10]:
#保存处理好的词汇
data = abstract_1000
f = open('abstract_1000.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

data = specific_1000
f = open('specific_1000.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

In [11]:
#abstract_words与词频
from collections import Counter
count = Counter(NN_words_new)
abstract_1000_count={}
specific_1000_count={}
missed_word_count={}
a_sum=0
s_sum=0
m_sum=0
for k, v in count.items():
    if k in abstract_1000:
        abstract_1000_count[k]=v
        a_sum=a_sum+v
    elif k in specific_1000:
        specific_1000_count[k]=v 
        s_sum=s_sum+v
    elif k in missed_words:
        missed_word_count[k]=v 
        m_sum=m_sum+v
print(a_sum,s_sum,m_sum)

64040 230825 134611


#词汇词云

In [12]:
#1000个抽象词汇的词云
text_long = '. '.join(abstract_1000)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('abstract_1000.png')

#1000个具体词汇的词云
text_long = '. '.join(specific_1000)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('specific_1000.png')

In [13]:
#missed_word的词云
text_long = '. '.join(missed_words)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('missed_words.png')
#abstract_words词云
text_long = '. '.join(abstract_words)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('abstract_words.png')

# 第二种分析方法
- 根据词频从大到小排序，从最开始选择最抽象的名词/最具体的词
- 什么样的词最抽象？
- 什么样的词最具体？

In [14]:
abstract=1.93
specific=4.82
abstract_words_1={}
specific_words_1={}
#按照值从大到小排序 
for k, v in sorted(count.items(),key=lambda x:x[1],reverse=True):
    if k in word_to_concreteness.keys():
        source = word_to_concreteness[k]
        #1000个抽象词
        if source<abstract:
            abstract_words_1[k]=v
        #1000个具体词
        if source>specific:
            specific_words_1[k]=v
print(len(abstract_words_1),len(specific_words_1))

1028 1078


In [15]:
abstract_1000_1=list(abstract_words_1.keys())[:1000]
specific_1000_1=list(specific_words_1.keys())[:1000]

#词云表示

In [16]:
abstract_list=[]
specific_list=[]
for word in NN_words_new:
    if word in abstract_1000_1:
        abstract_list.append(word)
    if word in specific_words_1:
        specific_list.append(word)

In [17]:
#保存处理好的词汇
data = abstract_1000_1
f = open('abstract_1000_1.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

data = specific_1000_1
f = open('specific_1000_1.txt','w')
f.write(str(data).encode('gbk','ignore').decode('gbk'))
f.close()

In [18]:
#抽象的词云
text_long = '. '.join(abstract_1000_1)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('abstract_1000_1.png')
#具体词云
text_long = '. '.join(specific_words_1)
w = wordcloud.WordCloud()
w.generate(text_long)
w.to_file('specific_words_1.png')

#两个方法的比较

In [19]:
#abstract_1000 vs specific_1000
#abstract_1000_1 vs specific_words_1
v1_a_diff=[]
v2_a_diff=[]

v1_s_diff=[]
v2_s_diff=[]
for word in set(abstract_words):
    if word in abstract_1000 and word not in abstract_1000_1:
        v1_a_diff.append(word)
    if word not in abstract_1000 and word in abstract_1000_1:
        v2_a_diff.append(word)
        
    if word in specific_1000 and word not in specific_1000_1:
        v1_s_diff.append(word)
    if word not in specific_1000 and word in specific_1000_1:
        v2_s_diff.append(word)
    
print(len(v1_a_diff),len(v2_a_diff))       
print(len(v1_s_diff),len(v2_s_diff))    

27 27
69 69


In [20]:
print(v1_a_diff)
print(v2_a_diff)

['persuasion', 'immortality', 'further', 'attentive', 'gingerly', 'overconfident', 'everyway', 'unfamiliar', 'intending', 'lackadaisical', 'lucky', 'potentially', 'best', 'proudly', 'necessitate', 'nonchalant', 'repugnant', 'befall', 'hesitance', 'misgiving', 'lording', 'compromising', 'dramatically', 'constancy', 'ritualistic', 'sentimentality', 'bias']
['understanding', 'awful', 'perseverance', 'coyly', 'deference', 'triumphing', 'whatnot', 'allure', 'liking', 'belonging', 'subjugation', 'involuntarily', 'dystopia', 'sophistication', 'considerate', 'decide', 'mundane', 'gallant', 'regression', 'funk', 'shrewd', 'elegancy', 'dimensionality', 'credence', 'anarchism', 'function', 'purer']


In [21]:
print(v1_s_diff)
print(v2_s_diff)

['mayo', 'toffee', 'doornail', 'goalpost', 'knapsack', 'antennae', 'waterbird', 'typewriter', 'corkscrew', 'jelly', 'crawlspace', 'mayonnaise', 'mower', 'ladle', 'guppy', 'cornea', 'javelin', 'treehouse', 'backboard', 'harmonica', 'tortellini', 'opossum', 'nightlight', 'scone', 'parakeet', 'yam', 'tweezer', 'anaconda', 'stethoscope', 'cheerleader', 'gardenia', 'mallet', 'sardine', 'speedboat', 'corkboard', 'crawfish', 'cookware', 'stirrup', 'macaroon', 'crate', 'jumprope', 'margarita', 'crayfish', 'diploma', 'teakettle', 'chipmunk', 'mantis', 'screwdriver', 'sketchpad', 'puma', 'hipbone', 'bedpan', 'honeybee', 'sandworm', 'hayfork', 'blowtorch', 'escalator', 'cookbook', 'trashcan', 'papaya', 'tinfoil', 'pepperoni', 'mistletoe', 'showerhead', 'dishtowel', 'softball', 'dustpan', 'scalpel', 'firetruck']
['sandbank', 'television', 'brick', 'marmalade', 'hookah', 'gun', 'claw', 'rancher', 'coke', 'headgear', 'dragonfly', 'cucumber', 'picnic', 'zipper', 'bonfire', 'woodstove', 'luggage', 'th

#第二种更好，出现更多次的词能够更好的学习到

In [22]:
import numpy as np
# 生成属性特征：#abstract_1000_1 vs specific_words_1
words = nltk.word_tokenize(text)

abstract_words_feat=np.zeros(1000)
specific_words_feat=np.zeros(1000)
for w in words:
    if w in abstract_1000_1:
        abstract_words_feat[abstract_1000_1.index(w)]=abstract_words_feat[abstract_1000_1.index(w)]+1
        print('A: ',w)
    if w in specific_1000_1:
        specific_words_feat[specific_1000_1.index(w)]=specific_words_feat[specific_1000_1.index(w)]+1
        print('B: ',w)

A:  very
A:  know


In [33]:
df = pd.read_csv(r"F:\dataset\Image\ArtEmis\artemis_official_data\official_data\test.csv")
text_list = df['utterance'].tolist()
text_list = [text.lower() for text in text_list]
text_list

get_NNwords(text_list[0])

['the', 'man', 'looks', 'sad', 'and', 'miserable', 'like', 'he', 'heard', 'some', 'terrible', 'news', 'that', 'shattered', 'his', 'very', 'soul']
[('the', 'DT'), ('man', 'NN'), ('looks', 'VBZ'), ('sad', 'JJ'), ('and', 'CC'), ('miserable', 'JJ'), ('like', 'IN'), ('he', 'PRP'), ('heard', 'VBD'), ('some', 'DT'), ('terrible', 'JJ'), ('news', 'NN'), ('that', 'WDT'), ('shattered', 'VBD'), ('his', 'PRP$'), ('very', 'NN'), ('soul', 'NN')]
man
news
very
soul


['man', 'news', 'very', 'soul']

In [37]:
import nltk
aft_tk=nltk.word_tokenize(text_list[0]) #分词后的列表
nltk_pos_tagged = nltk.pos_tag(aft_tk)
wnl = nltk.stem.WordNetLemmatizer()
aft_lem = [] #词形还原后的列表
for pstg in nltk_pos_tagged:
    word,tag = pstg
    if tag.startswith('NN'):
        aft_lem.append(wnl.lemmatize(word,'n'))
    elif tag.startswith('JJ'):
        aft_lem.append(wnl.lemmatize(word,'a'))
    elif tag.startswith('RB'):
        aft_lem.append(wnl.lemmatize(word,'r'))
    else :
        aft_lem.append(word)
print(aft_lem)

taged_sent = nltk.pos_tag(aft_lem)
taged_sent

['the', 'man', 'looks', 'sad', 'and', 'miserable', 'like', 'he', 'heard', 'some', 'terrible', 'news', 'that', 'shattered', 'his', 'very', 'soul']


[('the', 'DT'),
 ('man', 'NN'),
 ('looks', 'VBZ'),
 ('sad', 'JJ'),
 ('and', 'CC'),
 ('miserable', 'JJ'),
 ('like', 'IN'),
 ('he', 'PRP'),
 ('heard', 'VBD'),
 ('some', 'DT'),
 ('terrible', 'JJ'),
 ('news', 'NN'),
 ('that', 'WDT'),
 ('shattered', 'VBD'),
 ('his', 'PRP$'),
 ('very', 'NN'),
 ('soul', 'NN')]

In [38]:
text_list[0]

'the man looks sad and miserable like he heard some terrible news that shattered his very soul'

In [2]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


sentence = 'feet' # 举个例子
tokens = word_tokenize(sentence)  # 分词
tagged_sent = pos_tag(tokens)  # 获取单词词性

wnl = WordNetLemmatizer()
lemmas_sent = []
for tag in tagged_sent:
    wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
    lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos))  # 词形还原

print(" ".join(lemmas_sent))


foot


In [6]:
wnl.lemmatize('arthritis', pos=wordnet.NOUN)

'arthritis'

In [5]:
wordnet.NOUN

'n'

In [4]:
wordnet_pos

'n'

In [3]:
tag[0]

'feet'